# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [92]:
import re
import collections
import sys
import time
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [13]:


# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Let us shuffle the class labels.

In [15]:
## uncomment these lines if you have dowloaded the original file:
#np.random.seed(0)
#df = df.reindex(np.random.permutation(df.index))
#df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)

<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [56]:
stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Let's give it at try:

In [55]:
tokenizer('This :) is a <a> test! :-)</br>')

['test']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [19]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [20]:
#next(stream_docs(path='shuffled_movie_data.csv'))

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [21]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [22]:
#from sklearn.feature_extraction.text import HashingVectorizer
#vect = HashingVectorizer(decode_error='ignore', 
#                         n_features=2**21,
#                         preprocessor=None, 
#                         tokenizer=tokenizer)

# Exercise 1: define features based on word embeddings (pre-trained word2vec / Glove/Fastext emebddings can be used)
# Define suitable d dimension, and sequence length

In [23]:
doc_generator = stream_docs(path='shuffled_movie_data.csv')

In [24]:
def bag_words(gen, size):
    bag    = []
    for i, doc in enumerate(gen):
        bag += tokenizer(doc[0])
        sys.stdout.write('\r{:5.2f}%'.format(100*(i+1)/50000))
    sys.stdout.write('\rDone     \n\n')
    print('Bag Size: ', len(bag))
    count  = [['UNK', -1]]
    count += collections.Counter(bag).most_common(size - 1)

    dic    = {}
    for i, word in enumerate(count):
        dic[word[0]] = i
    
    return dic, dict(zip(dic.values(), dic.keys()))

In [25]:
dic, rev_dic = bag_words(doc_generator, 63069) #63069

Done     

Bag Size:  5978556


In [27]:
del doc_generator

In [103]:
class OneHotEncoder:
    def __init__ (self, rev_dict):
        self.size = len(rev_dict.values())
        self.dict = rev_dict
        
    def __repr__ (self):
        return ('OneHotEncoder Class (Size = {})'.format(self.n))
        
    def getVector(self, word):
        vec = np.zeros((1, self.size))
        vec[0, self.dict[word]] = 1
        return vec

In [172]:
class DocToVec:
    """
        Convert a list of words to feature vectors:
        
        Arguments:
            
            n_features: number of features per vector, by default 6818.
            fn        : tokenizer function
    """
    
    def __init__(self, dictionary, tokenizer):
        self.encoder  = OneHotEncoder(dic) 
        self.dict     = dictionary
        self.features = self.encoder.size
        self.fn       = tokenizer
        
    def transform(self, DocList):
        """
            Transform an array of list of words to a matrix:

            Arguments:

                DocList: Array of list of reviews

            Returns:

                Matrix with feature vectors of each review.

        """
    
        n   = len(DocList)
        vec = np.zeros(shape=(n, self.features), dtype = np.uint8)

        for idx, strn in enumerate(DocList):
            start_time = time.time()
            rev_tok    = self.fn(strn)
            veci = np.zeros((1, self.features))
            for word in rev_tok:
                if word in self.dict:
                    veci += self.encoder.getVector(word)
            vec[idx] = veci
            if ((idx+1) % 500 == 0):
                elapsed  = time.time() - start_time
                print('percent = {:4.1f}%, time estimated : {:3.1f} min'.format(100*(idx+1)/n, n*elapsed/60.0))
        return vec       

Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [173]:
vec = DocToVec(dic, tokenizer)

In [174]:
l = vec.transform([df.head().values[0][0]])

FirstReview = []
for i in range(63069):
    if l[0][i] != 0:
        FirstReview.append((rev_dic[i], int(l[0][i])))

FirstReview.sort(key=lambda x: x[1])

FirstReview = FirstReview[::-1]
for i in range(10):
    print('{:20} : {:3} times'.format(FirstReview[i][0], FirstReview[i][1]))

murder               :   5 times
detective            :   3 times
years                :   3 times
greenwich            :   2 times
disgrace             :   2 times
martha               :   2 times
investigation        :   2 times
committed            :   2 times
teenager             :   2 times
twenty               :   2 times


In [175]:
## Uncomment these lines if you don't save 'vec_data_X' and 'vec_data_y':
stream = stream_docs(path='shuffled_movie_data.csv')

X_train, y_train = get_minibatch(stream, size=50000)
y_train = np.asarray(y_train).reshape((-1, 1))

X_train = vec.transform(X_train)

np.save('vec_data_X', X_train)
np.save('vec_data_y', y_train)

percent =  1.0%, time estimated : 915.5 min
percent =  2.0%, time estimated : 339.3 min
percent =  3.0%, time estimated : 151.1 min
percent =  4.0%, time estimated : 238.5 min
percent =  5.0%, time estimated : 665.7 min
percent =  6.0%, time estimated : 517.4 min
percent =  7.0%, time estimated : 152.9 min
percent =  8.0%, time estimated : 330.7 min
percent =  9.0%, time estimated : 335.4 min
percent = 10.0%, time estimated : 223.6 min
percent = 11.0%, time estimated : 751.6 min
percent = 12.0%, time estimated : 373.0 min
percent = 13.0%, time estimated : 90.6 min
percent = 14.0%, time estimated : 215.8 min
percent = 15.0%, time estimated : 582.8 min
percent = 16.0%, time estimated : 696.4 min
percent = 17.0%, time estimated : 467.7 min
percent = 18.0%, time estimated : 339.7 min
percent = 19.0%, time estimated : 1473.2 min
percent = 20.0%, time estimated : 95.2 min
percent = 21.0%, time estimated : 539.1 min
percent = 22.0%, time estimated : 118.4 min
percent = 23.0%, time estimated :

In [134]:
#from sklearn.linear_model import SGDClassifier
#clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
#doc_stream = stream_docs(path='shuffled_movie_data.csv')
# Exercise 2: Define at least a Three layer neural network. Define its structure (number of hidden neurons, etc)
# Define a nonlinear function for hidden layers.
# Define a suitable loss function for binary classification
# Implement the backpropagation algorithm for this structure
# Do not use Keras / Tensorflow /PyTorch etc. libraries
# Train the model using SGD

In [102]:
#import pyprind
#pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    #pbar.update()

NameError: name 'doc_stream' is not defined

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [17]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))
#Exercise 3: compare  with your Neural Network

Accuracy: 0.867


I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [18]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [21]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

['./clf.pkl']

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [22]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Writing tokenizer.py


In [23]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

['./tokenizer.pkl']

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [24]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [25]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

array([0])

In [26]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

array([1])